# Market Amount Time Series Analysis

## Objective
- Identify time series inconsistencies in bond_data.market_amount
- Visualize negative values and abnormal jumps
- Determine specific correction strategies

## Expected Problem Patterns
1. **Negative values**: BOJ holdings > Cumulative issuance (purchase operations) → Normal state, acceptable
2. **Data gaps**: Incomplete data in bond_auction or boj_holdings
3. **Calculation timing issues**: Forward-fill logic problems

## 1. Environment Setup

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv
from supabase import create_client

# Set plot style (no Japanese font needed)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 10)

# Supabase connection
load_dotenv()
url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')
supabase = create_client(url, key)

print(f"✅ Supabase connected: {url[:30]}...")

## 2. Data Retrieval Functions

In [ ]:
def get_bond_timeseries(bond_code: str, start_date: str = None, end_date: str = None) -> pd.DataFrame:
    """
    Get time series data for a specific bond
    
    Args:
        bond_code: Bond code (9 digits)
        start_date: Start date (YYYY-MM-DD), optional
        end_date: End date (YYYY-MM-DD), optional
    
    Returns:
        DataFrame with columns: trade_date, bond_code, bond_name, market_amount, due_date
        If start_date and end_date are None, returns all available data from the earliest date
    """
    query = supabase.table('bond_data') \
        .select('trade_date, bond_code, bond_name, market_amount, due_date') \
        .eq('bond_code', bond_code) \
        .order('trade_date')
    
    if start_date:
        query = query.gte('trade_date', start_date)
    if end_date:
        query = query.lte('trade_date', end_date)
    
    result = query.execute()
    
    if not result.data:
        return pd.DataFrame()
    
    df = pd.DataFrame(result.data)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df['market_amount'] = pd.to_numeric(df['market_amount'], errors='coerce')
    
    print(f"Retrieved {len(df)} records from {df['trade_date'].min()} to {df['trade_date'].max()}")
    return df


def get_all_bond_codes_with_market_amount(limit: int = 100) -> list:
    """Get bond codes that have market_amount data"""
    result = supabase.table('bond_data') \
        .select('bond_code') \
        .not_.is_('market_amount', 'null') \
        .limit(limit) \
        .execute()
    
    if not result.data:
        return []
    
    bond_codes = list(set([row['bond_code'] for row in result.data]))
    return bond_codes


def get_auction_and_boj_data(bond_code: str) -> dict:
    """Get raw auction and BOJ holdings data for verification"""
    # Auction data
    auction = supabase.table('bond_auction') \
        .select('auction_date, allocated_amount') \
        .eq('bond_code', bond_code) \
        .order('auction_date') \
        .execute()
    
    # BOJ holdings data
    boj = supabase.table('boj_holdings') \
        .select('data_date, face_value') \
        .eq('bond_code', bond_code) \
        .order('data_date') \
        .execute()
    
    return {
        'auction': pd.DataFrame(auction.data) if auction.data else pd.DataFrame(),
        'boj': pd.DataFrame(boj.data) if boj.data else pd.DataFrame()
    }

print("✅ Data retrieval functions defined")

## 3. Anomaly Detection Functions

In [ ]:
def detect_anomalies(df: pd.DataFrame, threshold_pct: float = 20.0) -> pd.DataFrame:
    """
    Detect time series anomalies
    
    Args:
        df: Time series DataFrame (trade_date, market_amount)
        threshold_pct: Anomaly threshold (% change from previous day)
    
    Returns:
        DataFrame containing only anomalous data
    """
    df = df.sort_values('trade_date').copy()
    
    # Calculate day-over-day change
    df['pct_change'] = df['market_amount'].pct_change() * 100
    df['abs_change'] = df['market_amount'].diff()
    
    # Anomaly flags
    df['is_negative'] = df['market_amount'] < 0
    df['is_large_jump'] = df['pct_change'].abs() > threshold_pct
    df['is_anomaly'] = df['is_negative'] | df['is_large_jump']
    
    return df[df['is_anomaly']].copy()


def analyze_time_gaps(df: pd.DataFrame) -> pd.DataFrame:
    """Analyze data gap periods"""
    df = df.sort_values('trade_date').copy()
    df['date_diff'] = df['trade_date'].diff().dt.days
    
    # Detect gaps longer than 7 days (5 business days)
    gaps = df[df['date_diff'] > 7].copy()
    return gaps

print("✅ Anomaly detection functions defined")

## 4. Visualization Functions

In [ ]:
def plot_bond_timeseries(bond_code: str, figsize=(16, 10)):
    """
    Visualize bond time series in detail
    - Market amount
    - Cumulative issuance vs BOJ holdings
    - Day-over-day change rate
    """
    # Get data
    df = get_bond_timeseries(bond_code)
    raw_data = get_auction_and_boj_data(bond_code)
    
    if df.empty:
        print(f"No data found for: {bond_code}")
        return None
    
    # Detect anomalies
    anomalies = detect_anomalies(df)
    
    # Create plots
    fig, axes = plt.subplots(3, 1, figsize=figsize, sharex=True)
    
    # --- Subplot 1: Market Amount ---
    ax1 = axes[0]
    ax1.plot(df['trade_date'], df['market_amount'], 'b-', linewidth=1.5, label='Market Amount')
    ax1.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.7)
    
    # Highlight anomalies
    if not anomalies.empty:
        ax1.scatter(anomalies['trade_date'], anomalies['market_amount'], 
                   color='red', s=50, zorder=5, label='Anomalies')
    
    ax1.set_ylabel('Market Amount (100M JPY)', fontsize=12)
    ax1.set_title(f'{df.iloc[0]["bond_name"]} ({bond_code}) - Time Series Analysis', 
                 fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # --- Subplot 2: Cumulative Issuance vs BOJ Holdings ---
    ax2 = axes[1]
    
    if not raw_data['auction'].empty:
        auction_df = raw_data['auction'].copy()
        auction_df['auction_date'] = pd.to_datetime(auction_df['auction_date'])
        auction_df['cumulative'] = auction_df['allocated_amount'].cumsum()
        ax2.plot(auction_df['auction_date'], auction_df['cumulative'], 
                'g-', linewidth=1.5, label='Cumulative Issuance')
    
    if not raw_data['boj'].empty:
        boj_df = raw_data['boj'].copy()
        boj_df['data_date'] = pd.to_datetime(boj_df['data_date'])
        ax2.plot(boj_df['data_date'], boj_df['face_value'], 
                'r-', linewidth=1.5, label='BOJ Holdings')
    
    ax2.set_ylabel('Amount (100M JPY)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # --- Subplot 3: Day-over-day Change Rate ---
    ax3 = axes[2]
    df_copy = df.copy()
    df_copy['pct_change'] = df_copy['market_amount'].pct_change() * 100
    
    ax3.bar(df_copy['trade_date'], df_copy['pct_change'], 
           color='gray', alpha=0.7, width=1)
    ax3.axhline(y=20, color='red', linestyle='--', linewidth=1, alpha=0.5, label='±20% Threshold')
    ax3.axhline(y=-20, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    ax3.set_xlabel('Date', fontsize=12)
    ax3.set_ylabel('Change Rate (%)', fontsize=12)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'timeseries': df,
        'anomalies': anomalies,
        'raw_data': raw_data
    }

print("✅ Visualization functions defined")

## 5. Sample Bond Analysis (10-Year Bond #372)

In [ ]:
# Select sample bond (10-Year Bond #372)
SAMPLE_BOND_CODE = '003720067'

# Run time series analysis
result = plot_bond_timeseries(SAMPLE_BOND_CODE)

# Display anomaly summary
if result and not result['anomalies'].empty:
    print("\n=== Anomaly Summary ===")
    print(result['anomalies'][['trade_date', 'market_amount', 'pct_change', 'abs_change', 'is_negative', 'is_large_jump']])
else:
    print("\nNo anomalies detected")

## 6. Batch Analysis for Multiple Bonds

In [ ]:
def batch_anomaly_detection(bond_codes: list) -> pd.DataFrame:
    """Detect anomalies across multiple bonds"""
    all_anomalies = []
    
    for i, bond_code in enumerate(bond_codes):
        print(f"Analyzing {i+1}/{len(bond_codes)}: {bond_code}")
        df = get_bond_timeseries(bond_code)
        if df.empty:
            continue
        
        anomalies = detect_anomalies(df)
        if not anomalies.empty:
            anomalies['bond_code'] = bond_code
            all_anomalies.append(anomalies)
    
    if not all_anomalies:
        return pd.DataFrame()
    
    return pd.concat(all_anomalies, ignore_index=True)


# Example: Analyze first 20 bonds
bond_codes = get_all_bond_codes_with_market_amount(limit=100)
print(f"Total bonds with market_amount: {len(bond_codes)}")

batch_anomalies = batch_anomaly_detection(bond_codes[:20])  # Sample 20 bonds
print(f"\nTotal anomalies detected: {len(batch_anomalies)}")
if not batch_anomalies.empty:
    display(batch_anomalies.head(20))

## 7. Anomaly Pattern Classification

In [ ]:
def classify_anomaly_patterns(anomalies_df: pd.DataFrame) -> dict:
    """Classify anomaly patterns"""
    if anomalies_df.empty:
        print("No anomalies to classify")
        return {}
    
    summary = {
        'total_anomalies': len(anomalies_df),
        'negative_values': len(anomalies_df[anomalies_df['is_negative']]),
        'large_jumps': len(anomalies_df[anomalies_df['is_large_jump']]),
        'avg_pct_change': anomalies_df['pct_change'].mean(),
        'max_pct_change': anomalies_df['pct_change'].max(),
        'min_pct_change': anomalies_df['pct_change'].min()
    }
    
    print("=== Anomaly Pattern Classification ===")
    for key, value in summary.items():
        print(f"{key}: {value}")
    
    return summary


# Run pattern classification
if not batch_anomalies.empty:
    pattern_summary = classify_anomaly_patterns(batch_anomalies)

## 8. Analyze Specific Bond (Custom Input)

Use this cell to analyze any bond by entering its bond_code below.

In [ ]:
# Enter bond code here (9 digits)
CUSTOM_BOND_CODE = '003720067'  # Change this to analyze different bonds

# Run analysis
custom_result = plot_bond_timeseries(CUSTOM_BOND_CODE)

if custom_result:
    print(f"\n=== Analysis Results for {CUSTOM_BOND_CODE} ===")
    print(f"Total data points: {len(custom_result['timeseries'])}")
    print(f"Date range: {custom_result['timeseries']['trade_date'].min()} to {custom_result['timeseries']['trade_date'].max()}")
    print(f"Anomalies detected: {len(custom_result['anomalies'])}")
    
    if not custom_result['anomalies'].empty:
        print("\nAnomalies:")
        display(custom_result['anomalies'][['trade_date', 'market_amount', 'pct_change', 'is_negative', 'is_large_jump']])

## 9. Investigation Summary

### Findings
(Record analysis results here)

### Correction Strategy
(Record correction strategy here)

### Next Steps
1. Create list of problematic bonds
2. Correct calculation logic if needed
3. Re-calculate data